In [62]:
import pandas as pd
import sqlite3
from jinja2 import Template
import re
import numpy as np
from fuzzysearch import find_near_matches

# В данном алгоритме будем полагать, что текст произвольный, хоть у вас он немного причесан
df = pd.read_csv("test_data.csv")
#Приведем первые символы первого слова каждого предложения(реплики) к нижнему регистру
#Мы могли бы привести весь текст к нижнему регистру, но тогда потеряем информацию о собственных именах
df["text"] = df["text"].apply(lambda x: x[0].lower()+x[1:])
#Вставляем столбец insight для записи результатов парсинга
df["insight"]=""

#Подключаем Наташу
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    Doc,
    MorphVocab,
    NamesExtractor
) 

#Инициализируем Наташу
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

#Лемматизация текста
def lemmatize(text):
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  return [_.lemma for _ in doc.tokens] 

#Получение слов предложения
def get_all_words(str):
    doc = Doc(str)
    doc.segment(segmenter)
    return [token.text for token in doc.tokens]

#Скачиваем все русские имена
#Мужские
import urllib.request
destination = 'male_names.txt'
url = 'https://raw.githubusercontent.com/Raven-SL/ru-pnames-list/master/lists/male_names_rus.txt'
urllib.request.urlretrieve(url, destination)
file = open(destination, "r", encoding="utf-8")
male_names = [line[:-1].lower() for line in file.readlines()]
male_names = male_names[:-1]
#Женские
destination = 'female_names.txt'
url = 'https://raw.githubusercontent.com/Raven-SL/ru-pnames-list/master/lists/female_names_rus.txt'
urllib.request.urlretrieve(url, destination)
file = open(destination, "r", encoding="utf-8")
female_names = [line[:-1].lower() for line in file.readlines()]
female_names = female_names[:-1]
#Переделываем во множество
all_names = set(male_names + female_names)

#Функция поиска всех имен в предложении
def get_all_names(text):
  tokens = lemmatize(text)
  return set(tokens) & all_names

############# Основной класс для поиска инсайдов в диалоге ##################
# Основной функционал:
# Аналитический - поиск инсайдов разных типов
# Функциональный - позволяет быстрый и удобный доступ к разным видам инсайдов

class Dialog:
  #порядок поиска инсайдов если поиск происходит автоматически
  insights_find_order = [
    "greeting",
    "manager_name",
    "company",
    "goodbye"
  ]

  def __init__(self, df_):
    self.insight_table = None
    self.df = df_
    self.count = len(self.df)
    self.client_df = self.df[self.df["role"] == "client"]
    self.manager_df = self.df[self.df["role"] == "manager"] 
  
  #Возвращает инсайд по ключу
  def __getitem__(self, insight_type):
    #Возвращаеи инсайты типа insight_type
      #Песчитываем таблицу инсайдов
    self.get_insight_table()
    if not (self.insight_table is None):    
      table = self.insight_table.query('insight_name =="' + insight_type + '"')
      if len(table) > 0:
        return table["insight_value"].to_list()
    return None   
  
  #Находим все инсайты в порядке insights_find_order
  def find_all_insight(self):    
    for insight in Dialog.insights_find_order:
      self.find_insight(insight)

  def find_insight(self, type):
    if type == "greeting":
      self.find_greeting()
    elif type == "goodbye":
      self.find_goodbye()
    elif type == "manager_name":
      self.find_manager_name()   

  def set_insight(self, id, name, value):
    insight = self.df.loc[id, "insight"]
    if insight == "": 
      self.df.loc[id, "insight"] = name + "=" + value 
    else:
      self.df.loc[id, "insight"] = insight + "+" + name + "=" + value     
  
  #Создаем таблицу всех инсайтов
  def get_insight_table(self):    
    insight_messages = self.df.query('insight != ""')
    if len(insight_messages) == 0: return
    j = 0
    insight_dict ={}    
    for i in range(len(insight_messages)):
      #Учиваем случай что в одной и той же записи может быть несколько инсайдов
      msg = insight_messages.iloc[i]
      insights = msg["insight"].split("+")
      for k in range(len(insights)):
        #Будем формировать нашу таблицу из словаря
        insight_name = insights[k].partition("=")[0]
        insight_value = insights[k].partition("=")[2]
        insight_dict[j] = [msg["id"], insight_name, insight_value, msg["text"], msg["role"]]
        j = j+1    
    self.insight_table = pd.DataFrame.from_dict(insight_dict, orient='index', columns=["id", "insight_name", "insight_value", "text", "role"])
    return self.insight_table

  def get_insight_by_role(self, insight_type, role):
    #Возвращаеи инсайты типа insight_type с ролью - монеджер/клиент
      #Пересчитываем таблицу инсайдов
    self.get_insight_table()
    if not (self.insight_table is None):    
      table = self.insight_table.query(f"(insight_name == '{insight_type}') & (role == '{role}')")
      if len(table) > 0:
        return table["insight_value"].to_list()
    return None  

  #################################################################################################

  #Находим все приветствия
  def find_greeting(self):
    greeting_list = [
    "здравствуйте",
    "добрый день",
    "доброе утро",
    "добрый вечер",
    "приветствую",
    "привет",
    ]
    # Приветственные фразы будем искать в первых пяти репликах диалога, после Алло, Алло, можете говорить и т.п.
    #Функция определяющая содержит ли реплика приветственную фразу и записывающая информацию в поле insight реплики
    def check_greeting(message):      
      for greeting in greeting_list:
      #Нечеткий поиск с расстоянием Левенштейна = 1
        match = find_near_matches(greeting, message["text"], max_l_dist = 1)
        if match != []:
          self.set_insight(message["id"], "greeting", greeting)                       
      
    #Применяем функцию к первым 5 репликам менеджера
    self.manager_df.head().apply(check_greeting, axis = 1)  

  def find_goodbye(self):
    ### E.Извлекать реплики, где менеджер попрощался
    #Прощания в порядке убывания популярности
    goodbye_list = [
      "до свидания",
    "всего хорошего",
    "доброго дня",
    "доброй ночи",
    "счасливо",
    "удачного дня",
    "удачи",
    "прощайте"
    ]
    # Прощальные фразы будем искать в последних трех репликах диалога
    # Функция определяющая содержит ли реплика прощальную фразу
    def check_goodbye(message):
      for goodbye in goodbye_list:
        #Нечеткий поиск с расстоянием Левенштейна = 2
        match = find_near_matches(goodbye, message["text"], max_l_dist = 1)
        if match != []:
          self.set_insight(message["id"], "goodbye", goodbye)  

    #Применяем функцию к последним трем репликам менеджера
    self.manager_df.tail(n=3).apply(check_goodbye, axis = 1)
  
  # Функция находит все имена в репликах диалога
  def find_all_names(self):
    for i in range(self.count):
      names_ = get_all_names(self.df.iloc[i]["text"])
      if names_ != set(): 
        self.set_insight(self.df.iloc[i]["id"], "name", list(names_)[0])

  def find_manager_name(self):
    ##### Будем предполагать, что менеджер должен представиться за первые 5 своих реплик
    #Будем рассматривать разные виды диалогов от простого к сложному
    #1. Когда менеджер произносит фразу после которой всегда следует его Имя, например: меня зовут, мое имя
    #2. Когда менеджер произносит фразу после которой всегда следует его Имя, но может быть не сразу, например Вас приветствует
    # тут возможны варианты, Вас приветствует менеджер Татьяна, Вас приветствует Татьяна

    pre_name_phrase_list1 = [
      "меня зовут",
      "мое имя"
    ]

    pre_name_phrase_list2 = [
      "вас приветствует",
      "вы разговариваете",
      "вас беспокоит",
      "вам звонит"
    ]  

    def check_pre_name_phrase(message):
      text = message["text"]
      #Изначально предполагаем, что приветствия нет  
      matched = False
      for pre_name_phrase in pre_name_phrase_list1:
        #Нечеткий поиск с расстоянием Левенштейна = 1    
        match = find_near_matches(pre_name_phrase, text, max_l_dist = 1)
        if match != []:
          #Локализуем имя менеджера
          name = get_all_words(text[match[0].end:])[0]
          #Леммизируем имя менеджера
          name = lemmatize(name)[0]
          #Записываем инсайт в таблицу
          self.set_insight(message["id"], "manager_name", name) 
          matched = True            
          break
      if not matched:
        for pre_name_phrase in pre_name_phrase_list2:    
          match = find_near_matches(pre_name_phrase, text, max_l_dist = 1)
          if match != []:
            #Локализуем имя менеджера
            potential_name_container = text[match[0].end:]
            #Леммизируем предложение с именем менеджера
            potential_name_container = set(lemmatize(potential_name_container))
            #Вычисляем пересечение множества всех имен и потенциальных имен
            name = potential_name_container & all_names
            if name != set():
              #print(name)
              name = list(name)
              self.set_insight(message["id"], "manager_name", name[0]) 
              matched = True             
              break
    #Применяем функцию к первым 5 репликам менеджера
    self.manager_df.head().apply(check_pre_name_phrase, axis = 1)
    #Проверяем поймали ли мы наш инсайт
    if not(self["manager_name"] is None): return
    #Продолжаем искать имя нашего менеджера
    #Сейчас уже не будем отталкиваться от контекста, а будем смотреть на имена которые упоминают собеседники
    #Если менеджер представился, то наверняка за первые пять своих реплик, поэтому для начала найдем все имена которые есть в его пяти репликах
    names = set()
    for i in range(5):
      names = names | get_all_names(self.manager_df.iloc[i]["text"])
    names = list(names) 
    # Если имен в первых пяти репликах менеджера >= 2, то можно почти наверняка сказать, что он представился
    # тут есть два варианта, либо мы начинаем в искать более широкие контексты где мог представиться менеджер, либо
    # попробовать поискать упоминания его имени в более поздних репликах клиента, и отсутствия упоминания своего имени в своих репликах
    # так или иначе нам нужен новый тип инсайта - упоминание собственного имени в реплике
    # прогоним через него весь диалог и посмотрим кто кого упоминал
    for i in range(self.count):
      names_ = get_all_names(self.df.iloc[i]["text"])
      if names_ != set(): 
        self.set_insight(self.df.iloc[i]["id"], "name", list(names_)[0]) 


#################################### Функция принимает номер диалога и возвращает объект Dialog для анализа диалога

def GetDialog(dialog_number):
  dialog_df = df[(df["dlg_id"] == dialog_number)]
  dialog_df.insert(1, "id", dialog_df["line_n"])
  dialog_df.set_index("line_n", inplace=True)
  return Dialog(dialog_df)

################# Работа с объектом диалога 
#Создаем объект диалога
d = GetDialog(2)
#Находим все инсайты
d.find_all_insight()
#Складываем  их в таблицу к которой есть удобный доступ
d.get_insight_table()
#Например можно посмотреть значение конкретного инсайда просто обратясь по его ключу
d["greeting"]


['здравствуйте']